In [2]:
cd ../output/

/Users/martinscience/mount/work/utils/EBFilter/output


In [5]:
org_pileup = pd.read_csv('../../../../scratch/projects/somVar/pileup/test_D.pileup', sep='\t', dtype={0:str})
org_pileup[:30]

In [15]:
org_pileup.columns = ['Chr', 'Start', 'Ref', 'depth', 'read', 'qual']
org_pileup[:30]

,Chr,Start,Ref,depth,read,qual
0,1,12668,A,1,.,?
1,1,12669,A,1,.,A
2,1,12670,G,1,.,E
3,1,12671,A,1,.,A
4,1,12672,C,1,T,?
5,1,12673,G,1,.,D
6,1,12674,A,1,.,@
7,1,12675,C,1,.,B
8,1,12676,G,1,.,>
9,1,12677,G,1,.,D


In [7]:
import os
HOME = os.environ['HOME']

HOME = f"{HOME}/mount" # set HOME to run on different Macs

mut_file = pd.read_csv(f'{HOME}/scratch/projects/somVar/annovar/test_D-CR1.csv', sep='\t', dtype={'Chr':str})
indel = mut_file.query('Ref == "-" or Alt == "-"')
indel.to_csv('../test_D-indel.csv', sep='\t', index=False)

FileNotFoundError: [Errno 2] File b'/Users/martinscience/mount/scratch/projects/somVar/annovar/test_D-CR1.csv' does not exist: b'/Users/martinscience/mount/scratch/projects/somVar/annovar/test_D-CR1.csv'

In [8]:
merged_df = pd.read_csv('test_D-EB_144012.control.merged.csv', sep='\t')
indel_df = pd.read_csv('../test_D-indel.csv', sep='\t')
pileup_df = pd.read_csv('../test_D4.pileup', sep='\t', header=None, names=['Chr', 'Start', 'Ref', 'depth0', 'read0', 'Q0'])
target_df = pd.read_csv('test_D-EB_144012.target.pileup', sep='\t')

In [21]:
merged_pileup = pd.merge(org_pileup, target_df, how='inner', on=['Chr', 'Start'], sort=True, suffixes=('_org', '_target'))

In [25]:
merged_pileup.query('Start == 3395973')

,Chr,Start,Ref,depth,read,qual,depth0,read0,Q0
11,1,3395973,A,2,",-2cc.-2CC",?@,3,A-2NNa-2nnA-2NN,2?@


In [9]:
merged_df.query('Start == 190878538')

,Chr,Start,End,Ref,Alt,depth0,read0,Q0,depth1,read1,...,Q8,depth9,read9,Q9,depth10,read10,Q10,depth11,read11,Q11
1235,4,190878538,190878538,-,T,NaN,NaN,NaN,37,AAAAAAA+1TA+1TA+1TA+1TAA+1TA+1TA+1TA+1TAa+1tAA...,...,>_@@@?@?>??>???A>A>,54,AAAA+1TA+1TA+1TA+1TA+1TAA+1TA+1TA+1TA+1TAAAA+1...,_`@1@_:@@@@@<?@?<<^???????@???@?X@?X?@@_?AA?>?...,51,AAAA+1TAAAA+1TAA+1TAAAaAAAAAAAAaa+1ta+1tAAAa+1...,bb`AaaAAa?bAaA`AAA2baAABBa>@BB@B@@@B@@@@@@@@B@...,41,AAAA+1TAAAA+1TAA+1TAAAAAAAAAa+1tAaAAAAAAAAAAAA...,@__>`_@@@@_``@?_`@@@`A]_???_???????>=>>==


In [ ]:
pileup_control = 

In [21]:
indel_df.query('Chr == "4"')

,Chr,Start,End,Ref,Alt,Func.refGene,Gene.refGene,GeneDetail.refGene,ExonicFunc.refGene,AAChange.refGene,...,CADD_raw,CADD_phred,GERP++_RS,phyloP46way_placental,phyloP100way_vertebrate,SiPhy_29way_logOdds,dpsi_max_tissue,dpsi_zscore,ICGC_Id,ICGC_Occurrence
1191,4,85995,85995,-,A,exonic,ZNF595,NaN,unknown,UNKNOWN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1192,4,86101,86101,-,C,exonic,ZNF595,NaN,unknown,UNKNOWN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1193,4,87313,87313,-,A,exonic,ZNF595,NaN,unknown,UNKNOWN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1194,4,331693,331693,-,C,UTR5,ZNF141,NM_003441:c.-80_-79insC;NM_001348279:c.-80_-79...,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1195,4,843623,843623,C,-,intronic,GAK,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1196,4,1087168,1087168,-,AT,UTR3,RNF212,NM_001193318:c.*58_*59insAT,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1197,4,1737404,1737404,-,CT,intronic,TACC3,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1198,4,1739519,1739527,GCCATCCCT,-,intronic,TACC3,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MU858024,"BTCA-JP|2|239|0.00837,LINC-JP|1|244|0.00410,LI..."
1199,4,2439756,2439756,-,T,intronic,CFAP99,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1200,4,2965776,2965776,-,CGG,UTR5,GRK4,NM_001350173:c.-43762_-43761insCGG;NM_005307:c...,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
pileup_df.query('190878528 < Start < 190878548')

,Chr,Start,Ref,depth0,read0,Q0
8284557,4,190878529,N,3,TTT,BBB
8284558,4,190878530,N,3,AAA,??>
8284559,4,190878531,N,3,TTT,@@@
8284560,4,190878532,N,3,TTT,AAA
8284561,4,190878533,N,3,TTT,AAA
8284562,4,190878534,N,3,TTT,AAA
8284563,4,190878535,N,3,GGG,EEE
8284564,4,190878536,N,3,TTT,???
8284565,4,190878537,N,3,TTT,AAA
8284566,4,190878538,N,3,A+1TA+1TA+1T,???


In [19]:
target_df.query('Chr == "4"')

,Chr,Start,depth0,read0,Q0
304,4,85995,2,C+1AC+1A,gC
305,4,86101,4,G+1CG+1Cg+1cG+1C,EECi
306,4,87313,7,G+1AG+1Ag+1aG+1Ag+1aG+1Ag+1a,@bD?D?E
307,4,331693,18,G+1CG+1CG+1CG+1CG+1Cg+1cG+1CG+1CG+1CG+1CG+1CG+...,FiiEEBhEhhhhDhEEED
308,4,843622,8,aAAa-1nA-1NaA-1N*,>_S?A>@2
309,4,1087168,2,A+2ATA+2AT,BB
310,4,1737404,8,C+2CTC+2CTC+2CTC+2CTC+2CTC+2CTC+2CTC+2CT,DhDDCCCB
311,4,1739518,4,Cc-9nnnnnnnnnc-9nnnnnnnnnc-9nnnnnnnnn,b>??
312,4,2439756,22,C+1TC+1TC+1TC+1TC+1TC+1Tc+1tC+1TC+1TC+1TC+1TC+...,EhhCDhEgCDDfEEhhDCCghC
313,4,2965776,15,C$CccCC+3CGGC+3CGGcc+3tggC+3CGGc+3cggC+3CGGCCC,ihDCggDE5DEbhgC
